In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 10.3 MB/s eta 0:00:00


In [4]:
import torch
from torch import optim
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from torchvision import models
from utility.step_lr import StepLR


In [5]:
def train(swa_start, model, swa_model, epoch, device, train_loader, optimizer, log, scheduler, swa_scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        optimizer.zero_grad()

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions, targets, smoothing=0.001)
        loss.mean().backward()

        optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.get_last_lr()[0])

In [6]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets, smoothing=0.001)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [7]:
from torchvision import models
import torch.nn as nn

class ModifiedViT(nn.Module):
    def __init__(self, pretrained_model):
        super(ModifiedViT, self).__init__()
        self.pretrained = pretrained_model
        self.pretrained.head = nn.Identity()  # remove the existing linear layer
        self.new_head = nn.Sequential(
            nn.Linear(1000, 2),  # Adjust the input size to match the output size of the ViT model
        )

    def forward(self, x):
        x = self.pretrained(x)
        x = self.new_head(x)
        return x


In [8]:
if __name__ == "__main__":

    num_epochs = 50
    model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'ViT-swa-224'
    model_name = model_name_path + model_name
    initialize(seed=42)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    vit_b_16 = models.vit_b_16(pretrained=True)
    model = ModifiedViT(vit_b_16).to(device)
    model.eval()

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=224)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, weight_decay=0.1)

    swa_model = torch.optim.swa_utils.AveragedModel(model)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
    swa_start = 25
    swa_scheduler = torch.optim.swa_utils.SWALR(optimizer, swa_lr=0.0005)

    for epoch in range(num_epochs):
        train(swa_start, model, swa_model, epoch, device, dataset.train, optimizer, log, scheduler, swa_scheduler)
        test(model, device, dataset.valid, log)
        if epoch > swa_start:
          swa_model.update_parameters(model)
          swa_scheduler.step()
        else:
          scheduler.step()

    log.flush()
    # torch.save(model.state_dict(), '../model_trained/'+model_name+'.pth')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:00<00:00, 372MB/s]


┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      0.7515  │      0.40 %  ┃   1.000e-04  │       00:16  ┃      0.6926  │     52.33 %  ┃
┃           1  ┃      0.7167  │      0.47 %  ┃   9.990e-05  │       00:09  ┃      0.6648  │     57.33 %  ┃
┃           2  ┃      0.6859  │      0.55 %  ┃   9.961e-05  │       00:09  ┃      0.6411  │     62.67 %  ┃
┃           3  ┃      0.6587  │      0.61 %  ┃   9.911e-05  │       00:09  ┃      0.6192  │     67.67 %  ┃
┃           4  ┃      0.6340  │      0.66 %  ┃   9.843e-05  │       00:09  ┃      0.6004  │     74.00 %  ┃
┃           5  ┃      0.6115  │      0.70 %  ┃   9.755e-05  │       00:09  ┃      0.5831  │     77.33 %  ┃
┃           6  ┃      0.5910  │      0.73 %  ┃   9.649e-05  │       00:09  ┃      0.5681  │     79.33 %  ┃
┃           7  ┃      0.5723  │      0.75 %  ┃   9.524e-05  │       00:09  ┃      0.5539  │     80.67 %  ┃
┃           8  ┃      0.5547  │      0.78 %  ┃   9.382e-05  │       00:09  ┃      0.5403  │     83.33 %  ┃
┃           9  ┃      0.5381  │      

In [9]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [10]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)

tensor(0.9804, device='cuda:0')
tensor(0.9800, device='cuda:0')
tensor(0.9615, device='cuda:0')
tensor(0.9600, device='cuda:0')
tensor(1., device='cuda:0')


In [11]:
len(dataset.train)

9

In [12]:
len(dataset.test_x)

100